In [1]:
# Import necessary libraries

import PySimpleGUI as sg
import random

# Testing configuration
TESTING = True
FILL_TIMEOUT = 5

# Coin and selection lists
COIN_LIST = [
    ("$2", 200),
    ("$1", "Loonie"),
    ("25\u00A2", "Quarter"),
    ("10\u00A2", "Dime"),
    ("5\u00A2", "Nickel"),
    ("RETURN", "Return")
]

SELECTION_LIST = [
    ("CHIPS", "A"),
    ("CHOCOLATE", "B"),
    ("HEINEKEN", "C"),
    ("SUGAR DRINK", "D"),
    ("SURPRISE", "E")
]

PRICE_LIST = [
    ("150\u00A2", 150),
    ("100\u00A2", 400),
    ("50\u00A2", 500),
    ("40\u00A2", 600),
    ("30\u00A2", 700)
]

# List of items
ITEMS = [
    {'number': 0, 'name': 'CHIPS', 'price': 150},
    {'number': 2, 'name': 'CHOCOLATE', 'price': 100},
    {'number': 5, 'name': 'HEINEKEN', 'price': 50},
    {'number': 5, 'name': 'SUGAR DRINK', 'price': 40},
    {'number': 5, 'name': 'SURPRISE', 'price': 30},
]

# Value and name of coins
COINS_AMOUNT = [
    {'name': "$2", 'price': 200},
    {'name': "$1", 'price': 100},
    {'name': "25\u00A2", 'price': 25},
    {'name': "10\u00A2", 'price': 10},
    {'name': "5\u00A2", 'price': 5},
]

# Student number record database
student_number_record = {'1234': 0, '5678': 0}

# Function to print log messages during testing
def log(s):
    """Print the argument if testing/tracing is enabled."""
    if TESTING:
        print(s)

# Class for a simulated Temperature Sensor
class TemperatureSensor:
    def __init__(self):
        # Initialize your temperature sensor here (e.g., setup GPIO pins or connect to hardware).
        pass

    def read_temperature(self):
        # Implement code to read the temperature from your sensor.
        # Replace this with actual temperature reading logic.
        return random.randint(0, 26)  # For testing purposes, simulate temperature.

# State machine class
class VendingMachine(object):
    """Control a virtual hot beverage dispenser."""
    def __init__(self):
        # Initialize various state variables
        self.state = None  # current state
        self.states = {}  # dictionary of states
        self.event = ""  # no event detected
        self.hold_event = ""  # hold the event
        self.select = ""  # choice made by the user
        self.hold_event = ""  # hold the event
        self.product_price = 0
        self.Return = ""  # return key
        self.Sum_Of_Money = 0  # hold some of the money
        self.Return_Money = 0  # hold return money
        self.product_price = 0
        self.close_time = None
        self.text = ""  # create text for sg.text
        self.pop_text = ""  # create text for sg.popup
        self.name = ""  # using for changing color of sold-out item
        self.temperature_sensor = TemperatureSensor()  # Create an instance of the TemperatureSensor class.
        self.discount_valid = False
        self.student_number_valid = False
        self.student_number = ""
        # Initialize the fan state to be off
        self.fan_on = False

    def add_state(self, state):
        # Add a state to the dictionary of states
        self.states[state.name] = state

    def go_to_state(self, state_name):
        # Transition to a new state
        if self.state:
            log('Exiting %s' % (self.state.name))
            self.state.on_exit(self)
        self.state = self.states[state_name]
        log('Entering %s' % (self.state.name))
        self.state.on_entry(self)

    def update(self):
        # Update the current state
        if self.state:
            self.state.update(self)

    def update_window(self, window, text):
        # Update the text element in the GUI window
        window['-OUTPUT1-'].update(value=text)

    def update_text_element(self, window):
        # Update the text element in the GUI window
        self.update_window(window, self.text)

    def check_temperature(self):
        # Check the temperature using the temperature sensor
        temp = self.temperature_sensor.read_temperature()
        if temp > 25:
            self.turn_on_fan()
            print("Fan is ON")
        else:
            self.turn_off_fan()
            print("Fan is OFF")

    def turn_on_fan(self):
        # Turn on the fan if it's not already on
        if not self.fan_on:
            self.fan_on = True
            # Here, you can add code to actually turn on the fan (e.g., GPIO control).
            print("Turning on the fan")

    def turn_off_fan(self):
        # Turn off the fan if it's not already off
        if self.fan_on:
            self.fan_on = False
            # Here, you can add code to turn off the fan.
            print("Turning off the fan")
    ###
# States
###

class State():
    _NAME = ""  # Abstract class has no name

    def __init__(self):
        pass

    @property
    def name(self):
        return self._NAME

    def on_entry(self, machine):
        pass

    def on_exit(self, machine):
        pass

    def update(self, machine):
        pass

class AuthenticationState(State):
    """Authentication state."""
    _NAME = "authentication"

    def on_entry(self, machine):
        # Get student number input
        student_number = input("Enter your student number:")

        if student_number.isdigit():
            if student_number in student_number_record:
                # Successful authentication
                machine.text = "Authentication successful! Proceed to insert coin and select a product."
                machine.student_number_valid = True
                machine.student_number = student_number  # Store the student number

                # Increment the purchase count
                if machine.student_number in student_number_record:
                    student_number_record[machine.student_number] += 1
                else:
                    student_number_record[machine.student_number] = 0

                # Check if the purchase count is 3 for applying discount
                if student_number_record[machine.student_number] == 2:
                    machine.discount_valid = True
                    student_number_record[machine.student_number] = 0
                else:
                    machine.discount_valid = False

                machine.go_to_state("waiting")
            else:
                # Authentication failed
                machine.text = "Authentication failed. Please enter a valid student number."
                print("Authentication failed. Please enter a valid student number.")
                machine.student_number_valid = False
                machine.go_to_state("authentication")
        else:
            # Invalid input
            machine.text = "Invalid input. Please enter a numeric student number."
            print("Invalid input. Please enter a numeric student number.")
            machine.student_number_valid = False
            machine.update_text_element_threaded(window)
            machine.go_to_state("authentication")

class WaitingState(State):
    """Waiting for event."""
    _NAME = "waiting"

    def on_entry(self, machine):
        self.display_instructions()

    def update(self, machine):
        self.handle_coin_entry(machine)

    def display_instructions(self):
        print("Welcome to the Vending Machine!")
        print("Please insert coins to start.")

    def handle_coin_entry(self, machine):
        if machine.event != "":
            machine.hold_event = machine.event
            machine.go_to_state("selecting")

class SelectingState(State):
    """Waiting for event."""
    _NAME = "selecting"

    def on_entry(self, machine):
        self.process_coin_insertion(machine)

    def update(self, machine):
        self.process_coin_insertion_update(machine)

    def process_coin_insertion(self, machine):
        hold_event = machine.hold_event

        # Process the inserted coin
        if hold_event == 200:
            print("You entered 200 cents")
            machine.Sum_Of_Money += COINS_AMOUNT[0]['price']
        elif hold_event == "Loonie":
            print("You entered 100 cents")
            machine.Sum_Of_Money += COINS_AMOUNT[1]['price']
        elif hold_event == "Quarter":
            print("You entered 25 cents")
            machine.Sum_Of_Money += COINS_AMOUNT[2]['price']
        elif hold_event == "Dime":
            print("You entered 10 cents")
            machine.Sum_Of_Money += COINS_AMOUNT[3]['price']
        elif hold_event == "Nickel":
            print("You entered 5 cents")
            machine.Sum_Of_Money += COINS_AMOUNT[4]['price']
        else:
            machine.go_to_state('waiting')
            machine.text = "Welcome\n\nPlease Enter Money To Start"

        machine.text = f"Sum of money so far: {machine.Sum_Of_Money}"

    def process_coin_insertion_update(self, machine):
        if machine.hold_event != "":
            machine.hold_event = machine.event

        if machine.hold_event == "Return":
            machine.text = "Please wait, returning..."
            machine.go_to_state('return_and_receipt')
        else:
            machine.go_to_state('delivering')

class DeliveringState(State):
    _NAME = "delivering"

    def on_entry(self, machine):
        self.process_selection(machine)

    def update(self, machine):
        self.process_update(machine)

    def process_update(self, machine):
        if machine.hold_event != "":
            machine.hold_event = machine.event
        if machine.hold_event == "Return":
            machine.go_to_state('return_and_receipt')
        else:
            self.process_selection(machine)

    def process_selection(self, machine):
        item_names = [item['name'] for item in ITEMS]
        hold_event = machine.hold_event

        if hold_event in item_names:
            selected_item = next(item for item in ITEMS if item['name'] == hold_event)
            self.process_item(machine, selected_item)
        else:
            machine.go_to_state('selecting')

    def process_item(self, machine, selected_item):
        print(f"you selected the {selected_item['name']}")
        if selected_item['number'] > 0 and machine.Sum_Of_Money >= selected_item['price']:
            selected_item['number'] -= 1
            machine.product_price = selected_item['price']
            print("please receive Item")
            machine.go_to_state('return_and_receipt')
        elif selected_item['number'] == 0:
            machine.pop_text = "SOLD OUT!\n\nSelect a new item or Return"
        else:
            machine.pop_text = "Money is Missing! \n\nSelect a new item or Return"

class ReturnAndReceiptState(State):
    _NAME = "return_and_receipt"

    def on_entry(self, machine):
        self.calculate_return(machine)
        self.issue_receipt(machine)

    def calculate_return(self, machine):
        if machine.hold_event != "":
            # Check if discount is valid and apply it
            if machine.discount_valid:
                discount = 0.10  # 10% discount for repeat customers
                discounted_price = machine.product_price - (machine.product_price * discount)
                machine.product_price = discounted_price
                machine.text += f"\nRepeat customer discount applied: {discount * 100}%"
            machine.Return_Money = machine.Sum_Of_Money - machine.product_price
            print("Please receive your money back", machine.Return_Money)
            machine.text = f"Your Return Money is: {machine.Return_Money}\n\nThank you!"
            print(f"Your Return Money is: {machine.Return_Money}\n\nThank you!")
            machine.discount_valid = False
        else:
            machine.go_to_state('waiting')
            machine.Sum_Of_Money = 0

    def issue_receipt(self, machine):
        print("Name of product: " + machine.hold_event + "\n" +
              "The price you paid: " + str(machine.product_price) + "\n" +
              "Date: " + str(datetime.date.today()) + "\n" +
              "Time: " + str(datetime.datetime.now().time()) + "\n" +
              "Thank you!")
        sleep(0.2)
        machine.Sum_Of_Money = 0
        machine.go_to_state('authentication')

if __name__ == "__main__":
    # Define the coins column
    
    coin_col = []
    coin_col.append([sg.Text("ENTER COINS")])
    for item in COIN_LIST:
        button = sg.Button(item[0], key=item[1])  # Name and key are different
        row = [button]
        coin_col.append(row)

    # Define the selections column
    select_col = []
    select_col.append([sg.Text("SELECT ITEM")])
    for item in SELECTION_LIST:
        button = sg.Button(item[0], key=item[0])  # Use the name of the button as the key
        row = [button]
        select_col.append(row)
   # Define the price column     
    price_col = []
    price_col.append([sg.Text("PRICE")])
    for item in PRICE_LIST:
        button =sg.Text(item[0], font='Arial 14',text_color="pink")
        row = [button]
        price_col.append(row)
        
        
    # Define the layout as two columns separated by a vertical line
    layout = [ [sg.Column(coin_col, vertical_alignment="BOTTOM"),
                     sg.VSeparator(),
                     sg.Column(select_col, vertical_alignment="RIGHT"),
                    
                     sg.Column(price_col, vertical_alignment="LEFT")
                ],[sg.Text('Display Monitor')], [sg.Multiline("  Welcome\n\n  plese Enter Money To Start",size=(15,5) ,font='Courier 8', expand_x=True, expand_y=True,key='-OUTPUT1-')]
               ]
                 

    # Create the window
    window = sg.Window("Vending Machine", layout)
              
    vending = VendingMachine()

    # Add the states
    vending.add_state(WaitingState())    
    vending.add_state(SelectingState())  
    vending.add_state(DeliveringState())   
    vending.add_state(ReturnAndReceiptState())
    vending.add_state(AuthenticationState())
    
    
  # Set up the hardware button callback
    #key1.when_pressed = vending.button_action
    
    # Reset state is "waiting for choice"
    vending.go_to_state('authentication')
    
## Main loop ##          
    while True:
        
   #Set a Boolean variable to tell if the program is running on a Pi    
        #hardware_present = False
       # try:
            #from gpiozero import Button
            #key1 = Button(5)
            #hardware_present = True
        #except ModuleNotFoundError:
            #print("Not on a Raspberry Pi or gpiozero not installed.")
        vending.check_temperature() 
        event, values = window.read(timeout = 100, timeout_key = "__TIMEOUT__")
       
        if vending.pop_text!="":
            sg.popup_auto_close(vending.pop_text,title=None)
            vending.pop_text=""
        for item in ITEMS:
            if item['number']==0 :
                vending.name=item['name'] 
                for item in SELECTION_LIST:
                    if item[0]==vending.name:
                        window[item[0]].update(button_color = ('black on red '))
                

        if event == "__TIMEOUT__":
            pass  # no user interaction event occurred
            
        elif event == sg.WIN_CLOSED:
            print("Event sg.WIN_CLOSED")
            break
        else:
           
            vending.event=event 
            vending.update()
            
        #window['-OUTPUT1-'].update(vending.text)
        vending.update_text_element(window)
           
           
    window.close()
    print("Exiting...")

Entering authentication
Invalid input. Please enter a numeric student number.


AttributeError: 'VendingMachine' object has no attribute 'update_text_element_threaded'